In [ ]:
!pip install --upgrade pip

In [1]:
!pip install -U einops langchain pymilvus sentence-transformers openai

zsh:1: command not found: pip


In [ ]:
import os
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain_community.llms import VLLMOpenAI

In [ ]:
# Configurações do servidor de inferência e do modelo
INFERENCE_SERVER_URL = "http://vllm.llm-hosting.svc.cluster.local:8000/v1"
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
MAX_TOKENS = 1024
TOP_P = 0.95
TEMPERATURE = 0.01
PRESENCE_PENALTY = 1.03

In [ ]:
# Template do prompt
template = """<s>[INST] <<SYS>>
Você é um assistente prestativo, respeitoso e honesto chamado HatBot respondendo perguntas.
Você receberá uma pergunta que precisa responder com base no seu conhecimento.
Sempre responda da forma mais útil possível, enquanto se mantém seguro. Suas respostas não devem incluir conteúdo prejudicial, antiético, racista, sexista, tóxico, perigoso ou ilegal. Por favor, assegure-se de que suas respostas são socialmente imparciais e positivas.

Se uma pergunta não fizer sentido ou não for factualmente coerente, explique o motivo em vez de responder algo incorreto. Se você não souber a resposta para uma pergunta, por favor, não compartilhe informações falsas.
<</SYS>>

Pergunta: {question} [/INST]
"""

In [ ]:
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Configuração do modelo LLM
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=INFERENCE_SERVER_URL,
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    top_p=TOP_P,
    temperature=TEMPERATURE,
    presence_penalty=PRESENCE_PENALTY,
    streaming=True,
    verbose=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
# Função para consulta direta ao modelo
class DirectLLMQA:
    def __init__(self, llm, prompt_template):
        self.llm = llm
        self.prompt_template = prompt_template

    def run(self, question):
        prompt = self.prompt_template.format(question=question)
        response = self.llm.invoke({"query": prompt})
        return response

# Instancia o objeto de consulta direta ao LLM
direct_qa = DirectLLMQA(llm=llm, prompt_template=QA_CHAIN_PROMPT)

In [ ]:
# Exemplo de uso
def main():
    question = "Quais são os benefícios do uso de IA em ambientes empresariais?"
    response = direct_qa.run(question)
    print("Resposta do modelo:", response)

if __name__ == "__main__":
    main()
